In [1]:
import numpy as np
import pandas as pd
# library for splitting training-testing
from sklearn.model_selection import train_test_split
# library for classification
from sklearn.metrics import accuracy_score

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

from verstack import LGBMTuner

/home/jindalmadhur26/anaconda3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
2022-10-11 08:56:34.747767: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-11 08:56:34.747828: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# x_train = pd.read_csv("Data/train_allx.csv")
# x_test = pd.read_csv("Data/val_allx.csv")
# x_train.drop('ID', axis=1 , inplace=True)
# x_test.drop('ID', axis=1 , inplace=True)

# x_train = pd.read_csv("Data/train_x_null<50_notImputed.csv")
# x_test = pd.read_csv("Data/test_x_null<50_notImputed.csv")

# x_train = pd.read_csv("Data/train__mean_mode_Imputed.csv")
# x_test = pd.read_csv("Data/test__mean_mode_Imputed.csv")

# x_train = pd.read_csv("Data/train__Mean_MissingIndicator_Imputed.csv")
# x_test = pd.read_csv("Data/test__Mean_MissingIndicator_Imputed.csv")

x_train = pd.read_feather("Data/x_train__IterativeImputer_genFeats.ftr")
x_test = pd.read_feather("Data/x_test__IterativeImputer_genFeats.ftr")

# x_train = pd.read_feather("Data/x_train__groupbyInputed.ftr")
# x_test = pd.read_feather("Data/x_test__groupbyInputed.ftr")

y_train = pd.read_csv("Data/train_y.csv")
y_test = pd.read_csv("Data/Sample_Output.csv", names = ['ID', 'Target'], header=None)

In [3]:
# x_train = pd.get_dummies(x_train, columns=["D_36", "D_44"], prefix=["D_36", "D_44"], 
#                drop_first=True, dummy_na=True)

# x_test = pd.get_dummies(x_test, columns=["D_36", "D_44"], prefix=["D_36", "D_44"], 
#                drop_first=True, dummy_na=True)

# LightGBM

In [4]:
categorical_feats = x_train.select_dtypes('object').columns.tolist()
for c in categorical_feats:
    x_train[c] = x_train[c].astype('category')
#     x_val[c] = x_val[c].astype('category')
    x_test[c] = x_test[c].astype('category')

In [5]:
import lightgbm as lgb
print ('Training lightgbm')

# money
params = {
          "objective" : "multiclass",
          "num_class" : 4,
          "num_leaves" : 100,
          "max_depth": 8,
          "learning_rate" : 0.01,
          "bagging_fraction" : 0.9,  # subsample
          "feature_fraction" : 0.9,  # colsample_bytree
          "bagging_freq" : 5,        # subsample_freq
          "bagging_seed" : 2018,
          "verbosity" : -1,
          "metric" : ['auc_mu', 'multi_logloss'],
          "min_data_in_leaf" : 5000,
#           'categorical_feature' : x_train.select_dtypes('object').columns.tolist(),
#             'boosting' : 'dart',
            'num_threads' : 8,
#             'xgboost_dart_mode' : True,
            'is_unbalance' : True,
#             'class_weight' : 'balanced',
            'n_estimators' : 500
}

Training lightgbm


In [ ]:
tuner = LGBMTuner(metric='accuracy', trials=100, )
tuner.fit(x_train, y_train['Default_Flag'])


 * Initiating LGBMTuner.fit
     . Settings:
     .. Trying 100 trials
     .. Evaluation metric: accuracy 
     .. Study direction: minimize log_loss

     . Trial number: 0 finished
     .. Optimization score (lower-better): log_loss: 0.45087629180330663
     .. Evaluation score (greater-better): accuracy: 0.8168117912646323
 ...........................................................................
     . Trial number: 1 finished
     .. Optimization score (lower-better): log_loss: 0.4507632636836059
     .. Evaluation score (greater-better): accuracy: 0.8169163855694724
 ...........................................................................
     . Trial number: 2 finished
     .. Optimization score (lower-better): log_loss: 0.4543454486854394
     .. Evaluation score (greater-better): accuracy: 0.8162626711642218
 ...........................................................................
     . Trial number: 3 finished
     .. Optimization score (lower-better): log_loss: 0.

In [ ]:
tuner.plot_importances()

In [ ]:
tuner.plot_intermediate_values()

In [ ]:
tuner.plot_optimization_history()

In [ ]:
tuner.plot_param_importances()

In [ ]:
tuner.init_params

In [ ]:
tuner.best_params